In [1]:
import cv2
import numpy as np
from PIL import Image
import time

In [2]:
# 展示图片
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
starttime = time.time()

# 读取输入图片
ima = cv2.imread("zuo.jpg")
imb = cv2.imread("you.jpg")
imageA = cv2.resize(ima,(0,0),fx=0.2,fy=0.2)
imageB = cv2.resize(imb,(0,0),fx=0.2,fy=0.2)

In [4]:
# show('you', imageA)
# show('zuo', imageB)

In [5]:
# 检测A、B图片的SIFT关键特征点，并计算特征描述子
def detectAndDescribe(image):
    # 建立SIFT生成器
    sift = cv2.SIFT_create()
    # 检测SIFT特征点，并计算描述子
    (kps, features) = sift.detectAndCompute(image, None)
    # 在图像中标出关键点
    img_with_keypoints = cv2.drawKeypoints(image, kps, None)
#     show('keypoints', img_with_keypoints)
    # 返回特征点集，及对应的描述特征
    return (kps, features)

In [6]:
#检测A、B图片的SIFT关键特征点，并计算特征描述子
kpsA, featuresA = detectAndDescribe(imageA)
kpsB, featuresB = detectAndDescribe(imageB)

In [7]:
MIN = 10
FLANN_INDEX_KDTREE = 0

# 创建FLANN匹配器
indexParams = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
searchParams = dict(checks=50)
flann=cv2.FlannBasedMatcher(indexParams,searchParams)
match=flann.knnMatch(featuresA,featuresB,k=2)

In [8]:
good_match=[]
#过滤特征点
for i,(m,n) in enumerate(match):
    if(m.distance<0.75*n.distance):
        good_match.append(m)

# 绘制匹配结果
result = cv2.drawMatches(imageA, kpsA, imageB, kpsB, good_match, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
# show('match_res',result)
# cv2.imwrite('match_res.jpg', result)

In [9]:
# 当筛选后的匹配对大于10时，计算视角变换矩阵
if len(good_match) > MIN:
    src_pts = np.float32([kpsA[m.queryIdx].pt for m in good_match]).reshape(-1,1,2)
    ano_pts = np.float32([kpsB[m.trainIdx].pt for m in good_match]).reshape(-1,1,2)
    M,mask = cv2.findHomography(src_pts,ano_pts,cv2.RANSAC,5.0)
    warpImg = cv2.warpPerspective(imageB, np.linalg.inv(M), (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
    direct=warpImg.copy()
    direct[0:imageA.shape[0], 0:imageB.shape[1]] =imageA

# show('res',warpImg)
# show('direct', direct)
# cv2.imwrite('res.jpg', warpImg)
# cv2.imwrite('direct.jpg', direct)
rows,cols=imageA.shape[:2]
print(rows)
print(cols)

691
922


In [10]:
for col in range(0,cols):
    # 开始重叠的最左端
    if imageA[:, col].any() and warpImg[:, col].any():
        left = col
        print(left)
        break

for col in range(cols-1, 0, -1):
    #重叠的最右一列
    if imageA[:, col].any() and warpImg[:, col].any():
        right = col
        print(right)
        break

194
921


In [11]:
# 在重叠部位使用加权处理，减少两张图环境光带来的影响
res = np.zeros([rows, cols, 3], np.uint8)
for row in range(0, rows):
    for col in range(0, cols):
        if not imageA[row, col].any():  
            res[row, col] = warpImg[row, col]
        elif not warpImg[row, col].any():
            res[row, col] = imageA[row, col]
        else:
            srcImgLen = float(abs(col - left))
            testImgLen = float(abs(col - right))
            alpha = srcImgLen / (srcImgLen + testImgLen)
            res[row, col] = np.clip(imageA[row, col] * (1 - alpha) + warpImg[row, col] * alpha, 0, 255) #每个像素的取值限制在0-255

warpImg[0:imageA.shape[0], 0:imageA.shape[1]]=res
show('res',warpImg)
final=time.time()
print(final-starttime)

8.567049980163574


In [12]:
cv2.imwrite('finalres.jpg', warpImg)

True